# for each gage, filter by bfd == 1

In [3]:
import pandas as pd
import os

from main_jupyter import final_measurements_delta

# Define the directory containing the CSV files
directory = 'GSLB_ML'

# Initialize an empty DataFrame to store the results
compiled_data = pd.DataFrame(columns=['gage_id', 'date', 'q', 'bfd'])

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)

        # Read the CSV file
        df = pd.read_csv(file_path)

        # Filter rows where ML_BFD is 1
        filtered_df = df[df['ML_BFD'] == 1]

        # Extract gage_id from the filename (assuming filename is the gage_id)
        gage_id = os.path.splitext(filename)[0]

        # Add a new column for gage_id
        filtered_df['gage_id'] = gage_id

        # Select and rename the necessary columns
        filtered_df = filtered_df[['gage_id', 'date','Q', 'ML_BFD']]
        filtered_df.columns = ['gage_id', 'date', 'q', 'bfd']

        # Append to the compiled DataFrame
        compiled_data = pd.concat([compiled_data, filtered_df], ignore_index=True)

# Display the compiled DataFrame
compiled_data


/var/folders/fk/mzjttwrn7rn5nvcfhpgm4mrc0000gq/T/ipykernel_54062/2915055397.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['gage_id'] = gage_id
/var/folders/fk/mzjttwrn7rn5nvcfhpgm4mrc0000gq/T/ipykernel_54062/2915055397.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compiled_data = pd.concat([compiled_data, filtered_df], ignore_index=True)
/var/folders/fk/mzjttwrn7rn5nvcfhpgm4mrc0000gq/T/ipykernel_54062/2915055397.py:26: SettingWithCopyWarning: 
A value is trying to be set on a

,gage_id,date,q,bfd
0,10015900,1958-04-01,0.0,1.0
1,10015900,1958-04-02,0.0,1.0
2,10015900,1958-04-03,0.0,1.0
3,10015900,1958-04-04,0.0,1.0
4,10015900,1958-04-05,0.0,1.0
...,...,...,...,...
900051,10058600,1986-09-10,35.0,1.0
900052,10058600,1986-09-14,33.6,1.0
900053,10058600,1986-09-24,32.9,1.0
900054,10058600,1986-09-25,34.4,1.0


In [4]:
# Define the output directory
output_directory = 'base_on_bfd_1'
os.makedirs(output_directory, exist_ok=True)

# Define the output file path
output_file_path = os.path.join(output_directory, 'q_bfd_1.csv')

# Save the compiled DataFrame to a CSV file
compiled_data.to_csv(output_file_path, index=False)

# Display the path where the file is saved
output_file_path

'base_on_bfd_1/q_bfd_1.csv'

## pchip well data

In [6]:
well_ts = pd.read_csv('base_on_bfd_1/prep/GSLB_1900-2023_TS_with_aquifers.csv')

In [7]:
well_ts.head()

,AquiferID,Well_ID,Date,WTE,State
0,1,381033113480701,2012-09-06,7092.99,UT
1,1,381037113474001,2012-09-06,7175.95,UT
2,1,381152113442801,1995-11-22,6200.00,UT
3,1,381236113485601,2014-07-23,7151.00,UT
4,1,382113113435401,2008-09-03,5395.95,UT


In [8]:
well_info=pd.read_csv('base_on_bfd_1/prep/GSLB_1900-2023_wells_with_aquifers.csv')

In [9]:
well_info.head()

,Well_ID,Well_Name,lat_dec,long_dec,GSE,AquiferID,Aquifer_Name,State
0,381033113480701,(C-30-18)25aad- 1,38.175796,-113.802750,7098.0,1,GSL Basin,UT
1,381037113474001,(C-30-17)30bab- 1,38.176306,-113.795500,7193.0,1,GSL Basin,UT
2,381152113442801,(C-30-17)15cab- 1,38.197833,-113.741167,6550.0,1,GSL Basin,UT
3,381236113485601,(C-30-18)12cdb- 1,38.210028,-113.815500,7190.0,1,GSL Basin,UT
4,382113113435401,(C-28-17)22dda- 1,38.353571,-113.732473,5775.0,1,GSL Basin,UT


In [21]:
import pandas as pd
from scipy.interpolate import PchipInterpolator

# 将所有列名转换为小写
well_ts.columns = well_ts.columns.str.lower()
well_info.columns = well_info.columns.str.lower()

# 确保日期列转换为 datetime 格式
well_ts['date'] = pd.to_datetime(well_ts['date'])

# 统计每个井的数据点数量
well_counts = well_ts['well_id'].value_counts()
total_wells = len(well_counts)
wells_with_one_point = (well_counts == 1).sum()
wells_with_two_points = (well_counts == 2).sum()

# 打印统计信息
print(f"Total number of wells: {total_wells}")
print(f"Number of wells with only one data point: {wells_with_one_point}")
print(f"Number of wells with only two data points: {wells_with_two_points}")

# 对每个井进行 PCHIP 插值
interpolated_data = []
for well_id, group in well_ts.groupby('well_id'):
    if len(group) > 1:
        # 将日期转换为数值格式（时间戳）
        x = group['date'].map(pd.Timestamp.toordinal)
        y = group['wte']

        # 进行 PCHIP 插值
        interpolator = PchipInterpolator(x, y)
        group['wte'] = interpolator(x)
    else:
        # 如果只有一个数据点，直接复制
        group['wte'] = group['wte']
    interpolated_data.append(group)

# 将所有插值后的数据合并
interpolated_df = pd.concat(interpolated_data, ignore_index=True)

# 合并这些井的信息与 well_info 中的对应信息
merged_data = pd.merge(interpolated_df, well_info, on='well_id', how='left')

# 删除不需要的列
columns_to_drop = ['aquiferid_x', 'state_x', 'state_y', 'aquiferid_y', 'aquifer_name','well_name']
merged_data.drop(columns=columns_to_drop, inplace=True)

# 重命名列
merged_data.rename(columns={'lat_dec': 'well_lat', 'long_dec': 'well_lon'}, inplace=True)

# 显示合并后的数据
merged_data.head()


Total number of wells: 8602
Number of wells with only one data point: 4837
Number of wells with only two data points: 827


,well_id,date,wte,well_lat,well_lon,gse
0,381033113480701,2012-09-06,7092.99,38.175796,-113.802750,7098.0
1,381037113474001,2012-09-06,7175.95,38.176306,-113.795500,7193.0
2,381152113442801,1995-11-22,6200.00,38.197833,-113.741167,6550.0
3,381236113485601,2014-07-23,7151.00,38.210028,-113.815500,7190.0
4,382113113435401,2008-09-03,5395.95,38.353571,-113.732473,5775.0


In [22]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177209 entries, 0 to 177208
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   well_id   177209 non-null  int64         
 1   date      177209 non-null  datetime64[ns]
 2   wte       177209 non-null  float64       
 3   well_lat  177209 non-null  float64       
 4   well_lon  177209 non-null  float64       
 5   gse       177209 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 8.1 MB


In [23]:
merged_data.to_csv('base_on_bfd_1/well_pchip.csv', index=False)

# filter pchip by bfd == 1 dates

## 先用buffer筛选，得到gage-well的对应关系

In [25]:
gage_to_wells_df = pd.read_csv('data/gage_well_mapping_revised.csv')

In [26]:
gage_to_wells_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13984 entries, 0 to 13983
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   gage_id     13984 non-null  int64  
 1   well_id     13984 non-null  float64
 2   latitude    13984 non-null  float64
 3   longitude   13984 non-null  float64
 4   name        13984 non-null  object 
 5   River       13984 non-null  object 
 6   state       13984 non-null  object 
 7   Well_Name   13984 non-null  object 
 8   lat_dec     13984 non-null  float64
 9   long_dec    13984 non-null  float64
 10  Aquifer_Na  13984 non-null  object 
 11  State       13984 non-null  object 
dtypes: float64(5), int64(1), object(6)
memory usage: 1.3+ MB


In [27]:
import pandas as pd

# Assuming gage_to_wells_df is already loaded
# Convert all column names to lowercase
gage_to_wells_df.columns = gage_to_wells_df.columns.str.lower()

# Drop the specified columns
gage_to_wells_df.drop(columns=['name', 'river', 'state', 'aquifer_na'], inplace=True, errors='ignore')

# Rename the specified columns
gage_to_wells_df.rename(columns={
    'latitude': 'gage_lat',
    'longitude': 'gage_lon',
    'lat_dec': 'well_lat',
    'long_dec': 'well_lon'
}, inplace=True)

# Display the updated DataFrame
print(gage_to_wells_df.head())


    gage_id       well_id   gage_lat    gage_lon          well_name  \
0  10011500  4.055411e+14  40.965225 -110.853508  (A- 2-10) 9bbc- 1   
1  10015700  4.103411e+14  41.129114 -110.806563     13-119-34cbb02   
2  10015700  4.106111e+14  41.129114 -110.806563     13-119-13cb 01   
3  10015700  4.108211e+14  41.129114 -110.806563     13-119-02bb 01   
4  10015900  4.103411e+14  41.156058 -110.835176     13-119-34cbb02   

    well_lat    well_lon  
0  40.928003 -110.827951  
1  41.061336 -110.831286  
2  41.103003 -110.792952  
3  41.139114 -110.812119  
4  41.061336 -110.831286  


#### 筛选出buffer1内的wte

In [ ]:
final_measurements=merged_data

In [29]:
import pandas as pd

# Assuming well_pchip and gage_to_wells_df are already loaded
# Perform an inner merge to filter and combine the data based on 'well_id'
filtered_data = pd.merge(
    well_pchip,
    gage_to_wells_df[['gage_id', 'well_id', 'gage_lat', 'gage_lon', 'well_lat', 'well_lon']],
    on='well_id',
    how='inner'
)

# Drop the duplicate columns with suffixes
filtered_data.drop(columns=['well_lat_x', 'well_lon_x'], inplace=True)

# Optionally, rename the remaining columns if needed
filtered_data.rename(columns={'well_lat_y': 'well_lat', 'well_lon_y': 'well_lon'}, inplace=True)

# Display the new DataFrame
print(filtered_data.head())


           well_id        date     wte     gse   gage_id   gage_lat  \
0  393047114124001  1953-07-18  7117.0  7126.0  10172860  39.459528   
1  393358111515001  1962-02-19  4965.1  5022.0  10146000  39.713012   
2  393358111515001  1962-02-19  4965.1  5022.0  10146400  39.802454   
3  393358111515001  1962-02-19  4965.1  5022.0  10152000  40.150232   
4  393358111515001  1962-02-19  4965.1  5022.0  10167000  40.443838   

     gage_lon   well_lat    well_lon  
0 -114.023722  39.509387 -114.204445  
1 -111.804376  39.732733 -111.860768  
2 -111.862991  39.732733 -111.860768  
3 -111.726039  39.732733 -111.860768  
4 -111.922157  39.732733 -111.860768  


In [30]:
filtered_data.to_csv('base_on_bfd_1/after_buffer1.csv', index=False)

In [32]:
filtered_data = pd.read_csv('base_on_bfd_1/after_buffer1.csv')

In [33]:
# Calculate the number of unique wells
unique_wells_count = filtered_data['well_id'].nunique()

# Display the result
print(f"Number of unique wells: {unique_wells_count}")

unique_gages_count = filtered_data['gage_id'].nunique()
print(f"Number of unique gages: {unique_gages_count}")

Number of unique wells: 4770
Number of unique gages: 61


#### buffer2

In [36]:
import pandas as pd
import numpy as np

def filter_and_analyze_wte(filtered_data, merged_df, distance_buffer_meters=5, delta_bins=None):
    # Convert WTE from feet to meters
    filtered_data['wte_meters'] = filtered_data['wte'] * 0.3048

    # Merge filtered_data with merged_df to get reach_elev
    merged_data = pd.merge(
        filtered_data,
        merged_df[['well_id', 'reach_elev_m']],
        on='well_id',
        how='inner'
    )

    # Calculate the difference between reach elevation and WTE
    merged_data['delta_elev'] = merged_data['reach_elev_m'] - merged_data['wte_meters']

    # Filter out values where the difference is greater than the distance buffer
    filtered_result = merged_data[merged_data['delta_elev'] <= distance_buffer_meters]

    # Define default delta bins if not provided
    if delta_bins is None:
        delta_bins = [-float('inf'), -20, -10, -5, 0, 5, 10, 20, 30, 50, 75, 100, float('inf')]

    # Bin the delta_elev values
    bin_labels = [f"< {delta_bins[1]}"] + \
                 [f"{delta_bins[i]} to {delta_bins[i+1]}" for i in range(1, len(delta_bins)-2)] + \
                 [f">= {delta_bins[-2]}"]
    filtered_result['delta_bin'] = pd.cut(filtered_result['delta_elev'], bins=delta_bins, labels=bin_labels)

    # Calculate statistics
    total_measurements = len(filtered_result)
    dist_stats = filtered_result.groupby('delta_bin').size().reset_index(name='count')
    dist_stats['percentage'] = (dist_stats['count'] / total_measurements * 100).round(2)

    # Display statistics
    print(dist_stats)

    return filtered_result, dist_stats

# Example usage
filtered_data = pd.read_csv('base_on_bfd_1/after_buffer1.csv')  # Load your filtered data
merged_df = pd.read_csv('data/well_reach.csv')  # Load the merged data with reach_elev
filtered_result, dist_stats = filter_and_analyze_wte(filtered_data, merged_df, distance_buffer_meters=30)


     delta_bin  count  percentage
0        < -20  11521        7.27
1   -20 to -10  10517        6.63
2    -10 to -5  13760        8.68
3      -5 to 0  22492       14.18
4       0 to 5  27732       17.49
5      5 to 10  24979       15.75
6     10 to 20  31695       19.99
7     20 to 30  15876       10.01
8     30 to 50      0        0.00
9     50 to 75      0        0.00
10   75 to 100      0        0.00
11      >= 100      0        0.00


/var/folders/fk/mzjttwrn7rn5nvcfhpgm4mrc0000gq/T/ipykernel_54062/1356257825.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_result['delta_bin'] = pd.cut(filtered_result['delta_elev'], bins=delta_bins, labels=bin_labels)
/var/folders/fk/mzjttwrn7rn5nvcfhpgm4mrc0000gq/T/ipykernel_54062/1356257825.py:34: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dist_stats = filtered_result.groupby('delta_bin').size().reset_index(name='count')


In [37]:
filtered_result.head()

,well_id,date,wte,gse,gage_id,gage_lat,gage_lon,well_lat,well_lon,wte_meters,reach_elev_m,delta_elev,delta_bin
0,393047114124001,1953-07-18,7117.00,7126.0,10172860,39.459528,-114.023722,39.509387,-114.204445,2169.261600,2171.0,1.738400,0 to 5
408,393925111533301,1993-09-20,5074.74,5092.3,10146400,39.802454,-111.862991,39.656900,-111.893544,1546.780752,1548.0,1.219248,0 to 5
409,393925111533301,1993-09-20,5074.74,5092.3,10152000,40.150232,-111.726039,39.656900,-111.893544,1546.780752,1548.0,1.219248,0 to 5
410,393925111533301,1993-09-20,5074.74,5092.3,10167000,40.443838,-111.922157,39.656900,-111.893544,1546.780752,1548.0,1.219248,0 to 5
411,393926111533401,1949-04-15,5038.72,5092.0,10146400,39.802454,-111.862991,39.657178,-111.893822,1535.801856,1548.0,12.198144,10 to 20


In [38]:
dist_stats.head()

,delta_bin,count,percentage
0,< -20,11521,7.27
1,-20 to -10,10517,6.63
2,-10 to -5,13760,8.68
3,-5 to 0,22492,14.18
4,0 to 5,27732,17.49


In [39]:
filtered_result.to_csv('base_on_bfd_1/after_buffer2_30m.csv', index=False)

## use q dates to filter after buffer2

In [40]:
filtered_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158572 entries, 0 to 220535
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   well_id       158572 non-null  int64   
 1   date          158572 non-null  object  
 2   wte           158572 non-null  float64 
 3   gse           158572 non-null  float64 
 4   gage_id       158572 non-null  int64   
 5   gage_lat      158572 non-null  float64 
 6   gage_lon      158572 non-null  float64 
 7   well_lat      158572 non-null  float64 
 8   well_lon      158572 non-null  float64 
 9   wte_meters    158572 non-null  float64 
 10  reach_elev_m  158572 non-null  float64 
 11  delta_elev    158572 non-null  float64 
 12  delta_bin     158572 non-null  category
dtypes: category(1), float64(9), int64(2), object(1)
memory usage: 15.9+ MB


In [48]:
import pandas as pd

# Ensure both gage_id columns are of the same type
filtered_result['gage_id'] = filtered_result['gage_id'].astype(str)
compiled_data['gage_id'] = compiled_data['gage_id'].astype(str)

# Perform the merge
q_buffer2_pair = pd.merge(
    filtered_result,
    compiled_data[['gage_id', 'date','q','bfd']],
    on=['gage_id', 'date'],
    how='inner'
)

# Display the filtered DataFrame
q_buffer2_pair.head()


,well_id,date,wte,gse,gage_id,gage_lat,gage_lon,well_lat,well_lon,wte_meters,reach_elev_m,delta_elev,delta_bin,q,bfd
0,393925111533301,1993-09-20,5074.74,5092.3,10146400,39.802454,-111.862991,39.656900,-111.893544,1546.780752,1548.0,1.219248,0 to 5,6.8,1.0
1,393926111533401,1949-04-15,5038.72,5092.0,10167000,40.443838,-111.922157,39.657178,-111.893822,1535.801856,1548.0,12.198144,10 to 20,48.0,1.0
2,393926111533401,1951-10-18,5037.38,5092.0,10152000,40.150232,-111.726039,39.657178,-111.893822,1535.393424,1548.0,12.606576,10 to 20,50.0,1.0
3,393936111541801,1962-04-17,5034.30,5132.5,10167000,40.443838,-111.922157,39.659955,-111.905767,1534.454640,1548.0,13.545360,10 to 20,2.2,1.0
4,393936111541801,1963-04-24,5035.50,5132.5,10152000,40.150232,-111.726039,39.659955,-111.905767,1534.820400,1548.0,13.179600,10 to 20,13.0,1.0


In [49]:
q_buffer2_pair.to_csv('base_on_bfd_1/q_buffer2_pair.csv', index=False)

In [50]:
q_buffer2_pair.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46484 entries, 0 to 46483
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   well_id       46484 non-null  int64   
 1   date          46484 non-null  object  
 2   wte           46484 non-null  float64 
 3   gse           46484 non-null  float64 
 4   gage_id       46484 non-null  object  
 5   gage_lat      46484 non-null  float64 
 6   gage_lon      46484 non-null  float64 
 7   well_lat      46484 non-null  float64 
 8   well_lon      46484 non-null  float64 
 9   wte_meters    46484 non-null  float64 
 10  reach_elev_m  46484 non-null  float64 
 11  delta_elev    46484 non-null  float64 
 12  delta_bin     46484 non-null  category
 13  q             46484 non-null  float64 
 14  bfd           46484 non-null  float64 
dtypes: category(1), float64(11), int64(1), object(2)
memory usage: 5.0+ MB


# find the first bfd=1 wte

In [53]:
import pandas as pd

# Ensure 'date' is in datetime format
q_buffer2_pair['date'] = pd.to_datetime(q_buffer2_pair['date'])

# Sort by date to ensure chronological order
q_buffer2_pair = q_buffer2_pair.sort_values(by='date')

# Find the first occurrence of bfd=1 for each well and get the corresponding wte0 and q0
first_bfd_1 = q_buffer2_pair[q_buffer2_pair['bfd'] == 1].groupby('well_id').first().reset_index()

# Create a mapping of well_id to wte0 and q0
wte0_map = first_bfd_1.set_index('well_id')['wte'].to_dict()
q0_map = first_bfd_1.set_index('well_id')['q'].to_dict()

# Calculate delta_wte and delta_q directly in q_buffer2_pair
q_buffer2_pair['wte0'] = q_buffer2_pair['well_id'].map(wte0_map)
q_buffer2_pair['q0'] = q_buffer2_pair['well_id'].map(q0_map)

q_buffer2_pair['delta_wte'] = q_buffer2_pair['wte'] - q_buffer2_pair['wte0']
q_buffer2_pair['delta_q'] = q_buffer2_pair['q'] - q_buffer2_pair['q0']




In [55]:
q_buffer2_pair

,well_id,date,wte,gse,gage_id,gage_lat,gage_lon,well_lat,well_lon,wte_meters,reach_elev_m,delta_elev,delta_bin,q,bfd,wte0,q0,delta_wte,delta_q
46000,423438111453501,1928-08-23,5439.84,5516.94,10118000,41.834093,-112.055230,42.576867,-111.760789,1658.063232,1677.0,18.936768,10 to 20,311.00,1.0,5439.84,311.0,0.00,0.00
46017,423624111490101,1928-08-26,5440.89,5546.09,10118000,41.834093,-112.055230,42.606589,-111.817736,1658.383272,1682.5,24.116728,20 to 30,523.00,1.0,5440.89,523.0,0.00,0.00
34797,411605111481601,1932-08-08,4833.81,4840.00,10141000,41.278277,-112.091887,41.267997,-111.805216,1473.345288,1489.5,16.154712,10 to 20,38.00,1.0,4833.81,38.0,0.00,0.00
34557,411601111480701,1932-09-15,4821.94,4832.00,10141000,41.278277,-112.091887,41.266886,-111.802716,1469.727312,1489.5,19.772688,10 to 20,11.00,1.0,4821.94,11.0,0.00,0.00
35388,411641111472701,1932-09-20,4872.42,4880.00,10141000,41.278277,-112.091887,41.277997,-111.791605,1485.113616,1488.5,3.386384,0 to 5,38.00,1.0,4872.42,38.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19329,402946111233901,2023-08-31,5557.86,5700.00,10163000,40.237732,-111.699370,40.496083,-111.394861,1694.035728,1722.5,28.464272,20 to 30,120.00,1.0,5594.40,114.0,-36.54,6.00
18967,402937111214901,2023-08-31,5857.52,5880.00,10163000,40.237732,-111.699370,40.494306,-111.364528,1785.372096,1808.0,22.627904,20 to 30,120.00,1.0,5863.55,5.4,-6.03,114.60
34068,411544111461001,2023-09-07,4908.76,4923.52,10140100,41.254387,-111.855773,41.262222,-111.770278,1496.190048,1504.5,8.309952,5 to 10,36.90,1.0,4897.15,35.0,11.61,1.90
39678,414236112101201,2023-09-20,4296.82,4319.10,10126000,41.576321,-112.100782,41.709928,-112.170786,1309.670736,1321.0,11.329264,10 to 20,792.00,1.0,4296.64,511.0,0.18,281.00


In [56]:
# Drop the temporary columns if needed
q_buffer2_pair.drop(columns=['wte0', 'q0'], inplace=True)

# Display the resulting DataFrame with the new columns
q_buffer2_pair.head()

,well_id,date,wte,gse,gage_id,gage_lat,gage_lon,well_lat,well_lon,wte_meters,reach_elev_m,delta_elev,delta_bin,q,bfd,delta_wte,delta_q
46000,423438111453501,1928-08-23,5439.84,5516.94,10118000,41.834093,-112.055230,42.576867,-111.760789,1658.063232,1677.0,18.936768,10 to 20,311.0,1.0,0.0,0.0
46017,423624111490101,1928-08-26,5440.89,5546.09,10118000,41.834093,-112.055230,42.606589,-111.817736,1658.383272,1682.5,24.116728,20 to 30,523.0,1.0,0.0,0.0
34797,411605111481601,1932-08-08,4833.81,4840.00,10141000,41.278277,-112.091887,41.267997,-111.805216,1473.345288,1489.5,16.154712,10 to 20,38.0,1.0,0.0,0.0
34557,411601111480701,1932-09-15,4821.94,4832.00,10141000,41.278277,-112.091887,41.266886,-111.802716,1469.727312,1489.5,19.772688,10 to 20,11.0,1.0,0.0,0.0
35388,411641111472701,1932-09-20,4872.42,4880.00,10141000,41.278277,-112.091887,41.277997,-111.791605,1485.113616,1488.5,3.386384,0 to 5,38.0,1.0,0.0,0.0


In [57]:
q_buffer2_pair.to_csv('base_on_bfd_1/q_buffer2_pair_delta.csv', index=False)